## 1. Graph

In [ ]:
import networkx as nx

number_of_nodes = 10
erdos_prob = .2

graph = nx.erdos_renyi_graph(number_of_nodes, erdos_prob)
matrix = nx.to_numpy_matrix(graph).A

## 2.1. Dymanic Formulation

In [ ]:
def biochemical(n, x, adjecant, row_data):
    F = .05
    B = .002
    R = .003
    
    sigma = 0
    for j in range (0,n):
        sigma = sigma + R*(adjecant[j])*x*row_data[j]
    return (F - B*x - sigma)


In [ ]:
def birthdead(n, x, adjecant, row_data):
    B = .02
    R = .3
    a = 1
    b = 1
    
    sigma = 0
    for j in range (0,n):
        sigma = sigma + R*(adjecant[j])*pow(row_data[j],a)
    return (-B*pow(x,b) + sigma)

In [ ]:
def epidemic(n, x, adjecant, row_data):
    B = .02
    R = .3
    b = 1
    
    sigma = 0
    for j in range (0,n):
        sigma = sigma + R*(adjecant[j])*(1-x)*row_data[j]
    return (-B*pow(x,b) + sigma)

## 2.2. Generate Time-series

In [ ]:
import numpy as np

number_of_times = 1000
data = np.empty([number_of_times, number_of_nodes]) 
# todo: initial state?
data.fill(.5)

for t in range(0,number_of_times-1):
    for i in range(0,number_of_nodes):
        sigma = 0
        data[t+1,i] = biochemical(number_of_nodes, data[t,i], matrix[i], data[t])

## 3. Fit SINDy

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import odeint
from sklearn.linear_model import Lasso

import pysindy as ps

stlsq_optimizer = ps.STLSQ(threshold=.005, alpha=.5)

model = ps.SINDy(optimizer=stlsq_optimizer)
model.fit(data, t=1)
model.print()